In [3]:
# 대신증권 연결 확인
import win32com.client

instCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
print(instCpCybos.IsConnect)

import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from sklearn.preprocessing import minmax_scale
from statsmodels.tsa.stattools import coint
today = datetime.today().strftime("%Y%m%d") 
# start = (datetime.today() - timedelta()).strftime("%Y%m%d") 
# print(start, today)
pd.options.display.float_format = '{:.6f}'.format

1


In [4]:
# 분 차트 받아오기
def get_min(code,today,start):  # 종목, 기간, 오늘, 시점, 분, 시간간격
#     today = datetime.today().strftime("%Y%m%d") 
#     start = (datetime.today() - timedelta(j)).strftime("%Y%m%d") 
#     print(start, today)
    instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
    instStockChart.SetInputValue(0, code )
    instStockChart.SetInputValue(1, ord('1'))
    instStockChart.SetInputValue(2, start)
    instStockChart.SetInputValue(3, today)
    # instStockChart.SetInputValue(4, 1000)
    instStockChart.SetInputValue(5, (0,1,5))
    instStockChart.SetInputValue(6, ord('m'))  # 'm' : 분, 'T' : 틱
    instStockChart.SetInputValue(7, 1)      # 데이터 주기
    instStockChart.SetInputValue(9, ord('1'))
    instStockChart.SetInputValue(10, 3)

    instStockChart.BlockRequest()

    numData = instStockChart.GetHeaderValue(3)
    numField = instStockChart.GetHeaderValue(1) 

    temp = {}
    for i in range(numData):
        temp[str(instStockChart.GetDataValue(0, i)) +'.'+ str(instStockChart.GetDataValue(1, i))] = \
                [instStockChart.GetDataValue(2, i)]
    temp = pd.DataFrame(temp).transpose()
    temp.index.names = ['time']
    return temp


def one_day(temp,test_day):
    
    begin = np.where( np.array(temp.index) > str(test_day))[0][0]
    end = np.where( str(test_day+1) > np.array(temp.index) )[0][-1]
    print(begin-1)
    if len(temp.iloc[begin:end])==0:
        print(test_day,'는 공휴일 입니다.')
    else:
        return temp.iloc[begin-1:end]

# 이전 데이터 확인

In [30]:
kodex200 = pd.read_pickle('kodex200_1229')
kodex_x2 = pd.read_pickle('kodex_x2_1229')
kodex_inv = pd.read_pickle('kodex_inv_1229')
kodex_invx2 = pd.read_pickle('kodex_inversex2_1229')
kodex_active = pd.read_pickle('kodex_active_1229')
tiger_active = pd.read_pickle('tiger_active_1229')

# - 과거 1분단위 데이터 받아오기

In [19]:
# 이 부분만 바꿔주면 됨.
start = 20201224
finish = 20201229
before = '_1223'
now = '_1229'
########################


codes = {'A069500':'kodex200',
         'A114800':'kodex_inv'
         'A364690':'kodex_active',
         'A365040':'tiger_active',
         'A122630':'kodex_x2',
         'A252670':'kodex_inversex2'}


for code in list(codes.keys()):
    temp = None
    name = codes[code]
    temp1 = pd.read_pickle(name+before)
    temp2 = get_min(code,start,finish)[::-1]
    df = pd.concat([temp1,temp2])
    df.to_pickle(name+now)

In [55]:
kodex200 = pd.read_pickle('kodex200_1229')
kodex_x2 = pd.read_pickle('kodex_x2_1229')
kodex_invx2 = pd.read_pickle('kodex_inversex2_1229')
kodex_active = pd.read_pickle('kodex_active_1229')
tiger_active = pd.read_pickle('tiger_active_1229')
temp =  pd.merge(left = kodex200 , right = kodex_x2, how = "inner", on = "time")
temp =  pd.merge(left = temp , right = kodex_inv, how = "inner", on = "time")
temp =  pd.merge(left = temp , right = kodex_active, how = "inner", on = "time")
temp =  pd.merge(left = temp , right = tiger_active, how = "inner", on = "time")
temp.columns = ['kodex200','kodex_x2','kodex_invx2','kodex_active','tiger_active']
temp

,kodex200,kodex_x2,kodex_invx2,kodex_active,tiger_active
time,,,,,
20200929.901,31030,14695,4190,10040.000000,10100
20200929.902,31010,14685,4195,10020.000000,10100
20200929.903,31035,14700,4190,10015.000000,10100
20200929.904,31015,14690,4195,10005.000000,10100
20200929.905,31010,14685,4195,10000.000000,10100
20200929.906,30995,14670,4200,9995.000000,10095
20200929.907,31040,14710,4190,10010.000000,10095
20200929.909,31015,14685,4190,9995.000000,10080
20200929.912,31045,14715,4190,10010.000000,10090


In [41]:
def one_day(temp,test_day):
    
    begin = np.where( np.array(temp.index) > str(test_day))[0][0]
    end = np.where( str(test_day+1) > np.array(temp.index) )[0][-1]
    print(begin)
    if len(temp.iloc[begin:end])==0:
        print(test_day,'는 공휴일 입니다.')
    else:
        return temp.iloc[begin-1:end]